# **1. Execute the prescribed data preprocessing steps previously implemented in Assignment 3 to prepare the dataset for analysis.**

# **Installing GEOParse**

In [ ]:
!pip install GEOparse

# **Imports**

In [51]:
# All the neccessary imports
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import cross_val_score
import GEOparse
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV

# **Loading dataset**

In [ ]:
# 1. First, I will load the GSE45827 dataset using GEOparse package
gse = GEOparse.get_GEO(geo="GSE45827")

# **Data Cleaning**

In [ ]:
# 2. Then, I will do all the neccessary pre-processing

# This dataset contains 130 tumor samples as well as 11 normal tissues samples and 14 cell lines, for a total of 155 samples

dataframes = {} # a dictionary to save the sample name and its gene expression dataframe
for gsm_name, gsm in gse.gsms.items():
    df = gsm.table.copy() # the gene expression dataframe
    dataframes[gsm_name] = df # key = sample name and value = gene expression dataframe

# The below block of code converts a dictionary with key as sample name and value as dataframe to one dataframe where the column is the sample name and rows are the genes
df_list = [] # a list to save all dataframes, where each dataframe consists of 1 column (sample name) and 29873 rows (gene names), and the cells themselves are the expression values of the genes
for sample_name, df in dataframes.items(): # I loop over the dictionary I made
  df = df.set_index('ID_REF') # I change the gene expression dataframe (which is the value of the dictionary) such that its indices are the ID_REFs, which is the column used for gene name
  # This means that for every gene expression dataframe, it has 29873 rows which are the 29873 gene names
  df.columns = [sample_name] # Next, I manipulate the columns of the dataframe such that I only retain the sample_name

  # After all of these steps, I have a gene expression dataframe of 29873 gene names and 1 column, which is the sample name

  df_list.append(df) # I append my manipulated gene expression dataframe to my df_list

# df_list contains 155 dataframes, which corresponds for the 155 samples
# I want to merge them all in one big dataframe, so I use concat function across axis=1
final_df = pd.concat(df_list, axis=1)  # Now this is the final dataframe, which has as rows genes and as columns samples

final_df = final_df.T # I will transpose it to align with PCA later (such that features are columns and not rows)
final_df = final_df.rename_axis('Sample_Acc', axis='columns') # I change the axis (column) name such that it's Sample_Acc instead of ID_REF because I transposed, so the first column is for the sample names rather than gene names

# now, the dataframe needs an additional columns for the label. Therefore, I will make a dictionary where the key is the sample_acc and the value is the subtype

subtype_dict = {}
for gsm_name, gsm in gse.gsms.items():
    subtype = gsm.metadata['title'][0]  # Extract subtype
    Samples_index = subtype.find('Sample') - 1 # I need to retain the letters till the word "Sample", so I find its index
    subtype_dict[gsm_name] = subtype[:Samples_index] # I slice and add the subtype as a value for the sample accession key

# Now, subtype_dict contains the sample accession and the subtype..
# I want to use this information to add an additional for every row (every sample accession) with the corresponding label

# This code maps the accessions in the dataframe (with a certain order) to their subtypes from the subtype_dict
# The output is a list with subtypes with the exact order as the accessions of the dataframe
subtypes = []
for accession in final_df.index:
  subtypes.append(subtype_dict[accession])

final_df['Subtype'] = subtypes # I can simply now add a new column for the subtypes

# Now, FINALLY... I have a dataframe with 155 rows and 29874 columns (genes + label)!!

final_df.isna().sum().sum() # there are no nulls in expression values, so I don't remove anything...

final_df.columns.isna().sum() # no nulls in column names

final_df['Subtype'].isna().sum() # no nulls in labels

# Now, I want to remove any gene with expression of 0 in more than 25% of the samples

# Calculate the percentage of zero expression per gene (column)
zero_expression_percent = (final_df == 0).mean() * 100 # this is inspired by the following rationale
# final_df == 0 generates an array of booleans (TRUE, TRUE, FALSE, and so on)
# whenever a gene has zero expression, it is true, and whenever it is not 0, it is false
# True means 1, and false means 0
# therefore, if we have an array [TRUE, TRUE, FALSE] for example, it means that 66% of the data has zero expression
# This is calculated as follows: 1(TRUE, expression = 0) + 1(TRUE, expression=0) + 0(FALSE, expression != 0) / 3 = 0.66, which I multiply by 100 to be 66.66%
# Therefore, it is general that by getting the mean of true and falses, which are 1's and 0's, I can get the proportion of the samples that do not express the genes

# Now, I shall use this to keep only the genes with 0 expression in less than or equal to 25% of the samples

genes_to_keep = zero_expression_percent[zero_expression_percent <= 25].index
final_df = final_df[genes_to_keep]

# Apparently, nothing is removed, but it's okay :D

In [9]:
final_df

Sample_Acc,1007_s_at,1053_at,117_at,1294_at,1316_at,1405_i_at,1438_at,1487_at,1552256_a_at,1552257_a_at,...,89476_r_at,90265_at,90610_at,91617_at,91684_g_at,91703_at,91816_f_at,91826_at,91952_at,Subtype
GSM1116084,9.47065,8.36311,5.95426,6.02119,3.22997,10.82220,4.39698,6.65320,10.91320,10.07510,...,7.67662,8.58529,6.58532,6.47111,8.05685,5.51480,6.04568,5.90542,4.34839,Basal
GSM1116085,9.67440,8.72194,7.02523,7.24581,3.29352,9.29455,6.68936,7.20820,9.32204,9.34476,...,7.32080,6.21915,6.81551,6.54900,6.62661,5.58320,5.67661,6.68275,4.94622,Basal
GSM1116086,10.20800,7.78601,6.39671,6.85310,3.26204,9.45727,5.46440,6.68974,7.73131,8.43573,...,7.53787,7.86806,6.90606,6.32751,7.51148,5.85887,3.14257,7.77158,3.58973,Her2
GSM1116087,10.11420,9.44537,4.56023,5.42786,3.34766,11.51270,5.29748,6.87276,10.39210,9.63893,...,7.86515,8.79839,7.09756,6.03216,7.38772,4.55774,3.04537,6.47312,4.55167,Basal
GSM1116088,11.16360,7.71242,5.29008,7.51120,3.59086,8.83075,6.81821,6.45744,7.05836,9.37463,...,7.33174,7.01692,5.51992,6.09710,6.41229,5.34572,4.33680,8.48681,4.72397,Her2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM1116234,10.55470,6.88277,4.14597,6.37882,3.55309,6.36287,3.58838,6.44435,8.15810,6.76867,...,7.36935,7.40855,6.24758,5.05863,6.07391,4.92928,2.82099,7.16552,3.85854,Luminal A
GSM1116235,9.45290,6.83700,6.10766,6.99674,3.23899,9.56499,5.11154,5.82317,7.34009,8.50502,...,7.59543,8.17949,7.14055,5.46896,6.57521,5.46086,3.74439,5.70158,4.25533,Luminal A
GSM1116236,10.80010,7.29371,5.41169,8.37598,3.84879,8.11443,7.23567,6.02314,9.16813,8.68150,...,7.56045,8.15041,7.09543,6.06971,8.32561,4.97832,2.93524,7.88820,3.93143,Luminal B
GSM1116237,10.04640,7.71122,4.57742,6.56774,3.28091,8.07087,3.43906,8.44026,7.81914,8.75455,...,7.32272,6.97906,7.16655,6.46210,6.53545,4.98447,2.94121,7.27446,3.70314,Luminal B


# **Data Normalization**

In [8]:
# 3. I will normalize with the StandardScaler class.

# X contains all columns except the last one, which is the label
X = final_df.iloc[:, :-1]

# Y contains only the last column
Y = final_df.iloc[:, -1]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X) # where X is the dataframe containing the features

normalized_df = pd.DataFrame(data=X_scaled)
normalized_df['Subtype'] = Y.values
normalized_df.columns = final_df.columns

In [25]:
normalized_df

Sample_Acc,1007_s_at,1053_at,117_at,1294_at,1316_at,1405_i_at,1438_at,1487_at,1552256_a_at,1552257_a_at,...,89476_r_at,90265_at,90610_at,91617_at,91684_g_at,91703_at,91816_f_at,91826_at,91952_at,Subtype
0,-0.678000,0.599560,0.370553,-0.741989,-0.664745,1.024798,0.030256,0.061201,1.766899,1.679570,...,0.886933,2.098148,-1.376497,0.095778,1.218488,0.362477,1.774122,-0.624708,0.174815,Basal
1,-0.425216,0.953411,1.233018,0.356957,-0.540743,0.319727,1.331929,0.847399,0.389187,0.829193,...,-0.382264,-1.847092,-0.874469,0.225583,-0.413008,0.466291,1.503291,-0.051938,1.049811,Basal
2,0.236800,0.030467,0.726863,0.004548,-0.602168,0.394829,0.636364,0.112963,-0.988152,-0.229244,...,0.392017,0.902257,-0.676986,-0.143533,0.596376,0.884690,-0.356241,0.750360,-0.935574,Her2
3,0.120426,1.666803,-0.752076,-1.274431,-0.435102,1.343492,0.541583,0.372224,1.315703,1.171712,...,1.559413,2.453465,-0.259339,-0.635739,0.455202,-1.090104,-0.427568,-0.206402,0.472340,Basal
4,1.422375,-0.042102,-0.164319,0.595112,0.039442,0.105665,1.405093,-0.216107,-1.570828,0.863972,...,-0.343241,-0.516912,-3.700057,-0.527516,-0.657486,0.105855,0.520110,1.277373,0.724521,Her2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,0.666937,-0.860240,-1.085684,-0.421060,-0.034257,-1.033360,-0.428889,-0.234650,-0.618615,-2.170298,...,-0.209088,0.136082,-2.113083,-2.258143,-1.043481,-0.526198,-0.592223,0.303788,-0.542139,Luminal A
151,-0.700022,-0.905375,0.494088,0.133447,-0.647145,0.444546,0.436001,-1.114596,-1.326891,-0.148565,...,0.597331,1.421527,-0.165580,-1.574321,-0.471641,0.280609,0.085387,-0.774907,0.038611,Luminal A
152,0.971395,-0.455002,-0.066385,1.371147,0.542728,-0.224945,1.642138,-0.831325,0.255924,0.056921,...,0.472559,1.373040,-0.263984,-0.573161,1.525067,-0.451767,-0.508384,0.836291,-0.435456,Luminal B
153,0.036310,-0.043285,-0.738232,-0.251528,-0.565348,-0.245050,-0.513676,2.592701,-0.912104,0.141977,...,-0.375415,-0.580038,-0.108876,0.080763,-0.516996,-0.442433,-0.504003,0.384060,-0.769585,Luminal B


# **2. Implement Recursive Feature Elimination (RFE) with Support Vector Machine (SVM) as the estimator. Recursively select the most relevant features, and as part of this process, eliminate 50% of the least informative data.**

In [30]:
# My normalized dataset
X = normalized_df.iloc[:, :-1]
Y = normalized_df.iloc[:, -1]

# Create an SVM classifier
estimator = SVC(kernel="linear")

# Define the number of features to keep (50% of the features)
n_features_to_keep = int(0.5 * X.shape[1])

# Create the RFE selector
selector = RFE(estimator=estimator, n_features_to_select=n_features_to_keep, step=100) # by default step = 1, but it takes so much time, so I tried step = 100 to finish quicker.

# Fit the selector to your data
selector = selector.fit(X, Y)

# Get the selected features
selected_features = selector.support_

# Get the ranking of features (lower rank means more important)
feature_ranking = selector.ranking_

In [39]:
# Print the selected features and their ranking
print("Selected Features:", selected_features)
print("Feature Ranking:", feature_ranking)
sum((selected_features == True))

Selected Features: [False  True  True ...  True False False]
Feature Ranking: [ 90   1   1 ...   1  51 135]


14936

# **3. Perform Grid Search with a linear SVM using the selected features, exploring different values of the regularization parameter C (e.g., C = 1, 10, 100), and discuss your results.**

# **Robust classification function --> Can implement feature selection and may not, depending on the parameter**

In [67]:
def classifier(with_feature_selection = True):

  # Depending on the parameter, determine the dataframe to work with
  if with_feature_selection:
    # Create a new DataFrame with only the selected features
    selected_df = X.loc[:, selected_features]
  else:
    selected_df = X

  # Extract the label column
  labels = normalized_df.iloc[:, -1]

  # Split the data into training (80%) and testing (20%) sets
  X_train, X_test, y_train, y_test = train_test_split(selected_df, labels, test_size=0.2, random_state=42)

  # Define the range of C values to explore
  param_grid = {'C': [1, 10, 100]}

  # Create a linear SVM classifier
  svm_classifier = SVC(kernel="linear")

  # Perform Grid Search with cross-validation
  grid_search = GridSearchCV(estimator=svm_classifier, param_grid=param_grid, cv=5)

  # Fit the Grid Search to the training data with selected features
  grid_search.fit(X_train, y_train)

  # Get the best C value from the Grid Search
  best_C = grid_search.best_params_['C']

  # Train a linear SVM with the best C value on the entire training set
  final_svm_classifier = SVC(kernel="linear", C=best_C)
  final_svm_classifier.fit(X_train, y_train)

  # Evaluate the final model on the test set
  test_accuracy = final_svm_classifier.score(X_test, y_test)

  f1_scores = cross_val_score(final_svm_classifier, selected_df, labels, cv=5, scoring='f1_weighted')
  avg_f1_score = np.mean(f1_scores)

  return best_C, test_accuracy, f1_scores, avg_f1_score, X_train, y_train, X_test, y_test

# **Testing the accuracies before and after RFE**

In [75]:
classifier_vals_with_feature_selection = classifier(with_feature_selection = True)

print('Best C is: ', classifier_vals_with_feature_selection[0])
print('Test accuracy is: ', classifier_vals_with_feature_selection[1])

print('F1 scores are: ', classifier_vals_with_feature_selection[2])
print('The average F1 score is: ', classifier_vals_with_feature_selection[3])

######################################################

print()
print('Now, lets see the results without feature selection')
print('----------------------------')

classifier_vals = classifier(with_feature_selection = False)

print('Best C is: ', classifier_vals[0])
print('Test accuracy is: ', classifier_vals[1])

print('F1 scores are: ', classifier_vals[2])
print('The average F1 score is: ', classifier_vals[3])

Best C is:  1
Test accuracy is:  1.0
F1 scores are:  [1. 1. 1. 1. 1.]
The average F1 score is:  1.0

Now, lets see the results without feature selection
----------------------------
Best C is:  1
Test accuracy is:  0.9354838709677419
F1 scores are:  [0.96774194 0.90529256 0.96751635 0.96751635 0.90548162]
The average F1 score is:  0.9427097633549246


# **Checking the accuracies for all the C's before RFE**

In [78]:
# Create a dictionary to store the C values and their accuracies
accuracy_dict = {'C': [1, 10, 100], 'Accuracy': []}

# Calculate and store the accuracies for each C value
for c_value in accuracy_dict['C']:
    svm_classifier = SVC(kernel="linear", C=c_value)
    svm_classifier.fit(classifier_vals[4], classifier_vals[5])
    accuracy = svm_classifier.score(classifier_vals[6], classifier_vals[7])
    accuracy_dict['Accuracy'].append(accuracy)

# Convert the dictionary to a pandas DataFrame
accuracy_df = pd.DataFrame(accuracy_dict)

accuracy_df

,C,Accuracy
0,1,0.935484
1,10,0.935484
2,100,0.935484


# **Checking the accuracies for all the C's after RFE**

In [79]:
# Create a dictionary to store the C values and their accuracies
accuracy_dict = {'C': [1, 10, 100], 'Accuracy': []}

# Calculate and store the accuracies for each C value
for c_value in accuracy_dict['C']:
    svm_classifier = SVC(kernel="linear", C=c_value)
    svm_classifier.fit(classifier_vals_with_feature_selection[4], classifier_vals_with_feature_selection[5])
    accuracy = svm_classifier.score(classifier_vals_with_feature_selection[6], classifier_vals_with_feature_selection[7])
    accuracy_dict['Accuracy'].append(accuracy)

# Convert the dictionary to a pandas DataFrame
accuracy_df = pd.DataFrame(accuracy_dict)

accuracy_df

,C,Accuracy
0,1,1.0
1,10,1.0
2,100,1.0


# **Discussion**

1. Firstly, based on the preceding table, all C's have the optimal accuracies, whether it's before RFE or after RFE. This means that in this case, given the sample size and number of features, for this particular clean dataset, the parameters C does not affect the accuracy, if it's 1, 10, or 100. Maybe if it's 0.1 or 1000 it will differ.

2. Secondly, RFE increased the accuracy to 100%, while before RFE it was 93%. This means that RFE is good in choosing good and representative features